In [5]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [7]:
df1 = pd.read_csv('entrenar2_st_1820.csv')
df2 = pd.read_csv('entrenar2_st_1921.csv')
df3 = pd.read_csv('entrenar2_st_2022.csv')
df4 = pd.read_csv('entrenar2_st_2123.csv')
archivos_st = [df1,df2,df3,df4]

In [ ]:
xg_reg_st = xgb.XGBRegressor(objective ='reg:linear', 
            colsample_bytree = 1, learning_rate = 0.5,
            max_depth = 5, alpha = 10, n_estimators = 15) 
xg_reg_sc = xgb.XGBRegressor(objective ='reg:linear', 
            colsample_bytree = 0.5, learning_rate = 0.5,
            max_depth = 3,  n_estimators = 10) 

In [ ]:
rf_reg_st = RandomForestRegressor(n_estimators=40, max_depth=14, max_features=9,
                                   min_samples_split=7)
rf_reg_sc = RandomForestRegressor(n_estimators=40, max_depth=14, max_features=5,
                                   min_impurity_decrease=0.3, random_state=1)

In [ ]:
grd = GradientBoostingRegressor(n_estimators=20, max_depth=5, learning_rate=0.3,
                                    min_samples_split=2, min_samples_leaf=2,
                                    max_leaf_nodes=10)

In [3]:
def buscar_pesos(xg,rf,gb):
    suma_errores = 0
    for df in archivos_st:
        
        xg_reg_st = xgb.XGBRegressor(objective ='reg:linear', 
            colsample_bytree = 1, learning_rate = 0.5,
            max_depth = 5, alpha = 10, n_estimators = 15) 
        rf_reg_st = RandomForestRegressor(n_estimators=40, max_depth=14, max_features=9,
                                   min_samples_split=7)
        grd = GradientBoostingRegressor(n_estimators=20, max_depth=5, learning_rate=0.3,
                                    min_samples_split=2, min_samples_leaf=2,
                                    max_leaf_nodes=10)
        
        X,y = df.drop(['ref_hash','segundos'],axis=1),df['segundos']
        X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=0.1, random_state=150)

        xg_reg_st.fit(X_train,y_train)
        preds1 = xg_reg_st.predict(X_test)
        rf_reg_st.fit(X_train,y_train)
        preds2 = rf_reg_st.predict(X_test)
        grd.fit(X_train,y_train)
        preds3 = grd.predict(X_test)
        
        result = xg*preds1 + rf*preds2 + gb*preds3
        rmse = np.sqrt(mean_squared_error(y_test, result))
        suma_errores += rmse
        print('RMSE: %f' % (rmse))
       
    error_promedio = suma_errores / len(archivos_st)
    print('RMSE promedio: %f' % (error_promedio))
    return (error_promedio)

In [8]:
xg = 0.34
rf = 0.33
gb = 0.33
buscar_pesos(xg,rf,gb)

RMSE: 85778.443547
RMSE: 88161.128044
RMSE: 86874.604853
RMSE: 86316.015298
RMSE promedio: 86782.547935


86782.547935432056

In [9]:
xg = 0.20
rf = 0.60
gb = 0.20
buscar_pesos(xg,rf,gb)

RMSE: 85755.812770
RMSE: 88134.549420
RMSE: 86851.865733
RMSE: 86300.101885
RMSE promedio: 86760.582452


86760.582452111412

In [10]:
xg = 0.60
rf = 0.20
gb = 0.20
buscar_pesos(xg,rf,gb)

RMSE: 85817.766173
RMSE: 88170.295408
RMSE: 86901.147454
RMSE: 86351.942351
RMSE promedio: 86810.287846


86810.28784639205

In [11]:
xg = 0.20
rf = 0.20
gb = 0.60
buscar_pesos(xg,rf,gb)

RMSE: 85826.557664
RMSE: 88216.380975
RMSE: 86933.730362
RMSE: 86358.249010
RMSE promedio: 86833.729503


86833.729502841859

In [12]:
t_xg = pd.read_csv('target_xgboost.csv')
t_rf = pd.read_csv('target_randomForest.csv')
t_gb = pd.read_csv('target_gradient.csv')

In [14]:
target = pd.DataFrame()
target['ref_hash'] = t_xg['ref_hash']
target.head()

,ref_hash
0,1000169251625791246_sc
1,1000169251625791246_st
2,1000395625957344683_sc
3,1000395625957344683_st
4,1003027494996471685_sc


In [15]:
target['obj'] = 0.2*t_xg['obj']+0.6*t_rf['obj']+0.2*t_gb['obj']
target.head()

,ref_hash,obj
0,1000169251625791246_sc,248412.406944
1,1000169251625791246_st,63698.041881
2,1000395625957344683_sc,132223.796548
3,1000395625957344683_st,60543.855606
4,1003027494996471685_sc,244833.413795


In [16]:
target.to_csv('target_ponderado.csv', index=False)